# Imports

data sources: https://github.com/CSSEGISandData/COVID-19_Unified-Dataset documentation: https://github.com/OxCGRT/covid-policy-tracker/blob/master/documentation/codebook.md

In [251]:
import datetime
import pandas as pd
import numpy as np
import pyreadr
from sklearn.preprocessing import OneHotEncoder

# model db with 30 features

## Deltas

In [368]:
lma_baseline=pd.read_csv('/Users/paulcharpentier/code/TristanBOOK/coronanet/coronanet/data/LMA_relative_time_cases.csv')
lma_baseline['Cases']=lma_baseline['Cases'].astype(float)
lma_baseline=lma_baseline.drop('Unnamed: 0',axis=1)

J_14=(lma_baseline['Cases']/lma_baseline['J-14'])**(1/13)
J14=(lma_baseline['J14']/lma_baseline['Cases'])**(1/13)
J7=(lma_baseline['J7']/lma_baseline['Cases'])**(1/6)
J21=(lma_baseline['J21']/lma_baseline['Cases'])**(1/20)

lma_baseline['Delta7']=(J7/J_14)
lma_baseline['Delta14']=(J14/J_14)
lma_baseline['Delta21']=(J21/J_14)

In [369]:
delta_30=lma_baseline[['Date','ID','Policy','Delta7','Delta14','Delta21']]
delta_30['Country']=delta_30['ID']
delta_30.set_index('Date',inplace=True)

/Users/paulcharpentier/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [370]:
delta_30.head()

,ID,Policy,Delta7,Delta14,Delta21,Country
Date,,,,,,
2020-01-26,CN,C1,NaN,NaN,NaN,CN
2020-01-26,CN,C2,NaN,NaN,NaN,CN
2020-01-22,CN,C3,NaN,NaN,NaN,CN
2020-01-22,CN,C4,NaN,NaN,NaN,CN
2020-01-23,CN,C5,NaN,NaN,NaN,CN


## 30 features db

In [371]:
policies= pyreadr.read_r('/Users/paulcharpentier/Desktop/Coronanet_files/Policy.RData')
policies_df = pd.DataFrame(policies['Policy'])
p_df=policies_df
p_df=p_df.rename(columns={"ID": "Country"})
clean_df=p_df[p_df['PolicyType']=='C1'][["Country","Date"]]
col_names=list(p_df.PolicyType.unique())

for c in col_names:
    clean_df[c]=p_df[p_df['PolicyType']==c].PolicyValue.values

clean_df=clean_df.drop("ID",axis=1)
clean_df=clean_df.drop("IC",axis=1)
clean_df=clean_df.rename(columns={"Country": "ID"})
clean_df.set_index('Date',inplace=True)

In [372]:
clean_df.head()

,ID,C1,C2,C3,C4,C5,C6,C7,C8,E1,...,I1D,I2,I2D,I3,I3D,I4,I4D,I4L,I4LD,M1
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-01,AD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2020-01-02,AD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2020-01-03,AD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2020-01-04,AD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2020-01-05,AD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


## Merge db

In [373]:
df=clean_df
df=df.merge(delta_30,on=['Date','ID'],how='right')

In [374]:
df

,ID,C1,C2,C3,C4,C5,C6,C7,C8,E1,...,I4,I4D,I4L,I4LD,M1,Policy,Delta7,Delta14,Delta21,Country
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-26,CN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,C1,NaN,NaN,NaN,CN
2020-01-26,CN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,C2,NaN,NaN,NaN,CN
2020-01-22,CN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,C3,NaN,NaN,NaN,CN
2020-01-22,CN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,C4,NaN,NaN,NaN,CN
2020-01-23,CN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,C5,NaN,NaN,NaN,CN
2020-01-23,CN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,C6,NaN,NaN,NaN,CN
2020-01-23,CN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,C7,NaN,NaN,NaN,CN
2020-02-25,CN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,C8,0.950782,0.946824,0.944008,CN
2020-04-11,CN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,E1,1.054934,1.024427,1.015459,CN


In [360]:
d=df[['ID','Policy','Delta7','Delta14','Delta21']]
f=d.dropna()
f=f[f['Delta7']>0]
f=f[f['Delta7']!=1]
f=f[f['Delta7']<100]
ts=f
dum=pd.get_dummies(ts.Policy)
train_set=dum.merge(f, on='Date')

In [363]:
X=train_set.drop(['Policy','Delta7','Delta14','Delta21','ID'],axis=1)
X.reset_index(drop=True, inplace=True)

      C1  C2  C3  C4  C5  C6  C7  C8  E1  E2  E3  E4  H1  H2  H3  H4  H5  H6  \
0      0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   
1      0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   
2      0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   
3      0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   
4      0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
5      0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
6      0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
7      0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   
8      0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   
9      0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   
10     0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   
11     0   0   0   0   0   0   0   0   0

In [ ]:
X=train_set[]

# model db with 125 features

In [252]:
dataR= pyreadr.read_r('/Users/paulcharpentier/Desktop/Coronanet_files/COVID-19.RData')
cases_df = pd.DataFrame(dataR['COVID19'])
cases_df.head()

,ID,Date,Cases,Cases_New,Type,Age,Sex,Source
0,AD,2020-01-22,0,0,Active,Total,Total,JHU
1,AD,2020-01-22,0,0,Confirmed,Total,Total,JHU
2,AD,2020-01-22,0,0,Deaths,Total,Total,JHU
3,AD,2020-01-22,0,0,Recovered,Total,Total,JHU
4,AD,2020-01-23,0,0,Active,Total,Total,JHU


In [253]:
data_lma=pd.read_csv('https://raw.githubusercontent.com/TristanBOOK/coronanet/master/coronanet/data/LMA_Data_10_semaines_and_Pol_fam.csv')

In [254]:
data_lma.head()

,ID,Date,PolicyFamily,PolicyType,value,Cases,Cases_New,Type,Source,J-28,J-21,J-14,J-7,J0,J7,J14,J21,J28,J35,J42
0,AD,2020-02-27,E,E3_diff,141399008,0.0,0.0,Deaths,JHU,0,0,0,0,0,0,0,0,3,15,25
1,AD,2020-02-28,E,E3_diff,-141399008,0.0,0.0,Deaths,JHU,0,0,0,0,0,0,0,0,3,16,26
2,AD,2020-03-02,H,H2_diff,1,0.0,0.0,Deaths,JHU,0,0,0,0,0,0,0,1,8,21,29
3,AD,2020-03-11,H,H6_diff,2,0.0,0.0,Deaths,JHU,0,0,0,0,0,0,1,14,23,33,37
4,AD,2020-03-13,C,C3_diff,1,0.0,0.0,Deaths,JHU,0,0,0,0,0,0,3,16,26,35,40


## Policies DF

In [255]:
policies= pyreadr.read_r('/Users/paulcharpentier/Desktop/Coronanet_files/Policy.RData')
policies_df = pd.DataFrame(policies['Policy'])
policies_df = policies_df.rename(columns={"ID": "Country"})

In [256]:
policies_df.head()

,Country,Date,PolicyType,PolicyValue,PolicyFlag,PolicyNotes,PolicySource
0,AD,2020-01-01,C1,0.0,NaN,NaN,OxCGRT
1,AD,2020-01-01,C2,0.0,NaN,NaN,OxCGRT
2,AD,2020-01-01,C3,0.0,NaN,NaN,OxCGRT
3,AD,2020-01-01,C4,0.0,NaN,NaN,OxCGRT
4,AD,2020-01-01,C5,0.0,NaN,NaN,OxCGRT


In [257]:
policies_df.set_index('PolicyType',inplace=True)
policies_df=policies_df.drop("I1",axis=0)
policies_df=policies_df.drop("I2",axis=0)
policies_df=policies_df.drop("I3",axis=0)
policies_df=policies_df.drop("I4",axis=0)
policies_df=policies_df.drop("IC",axis=0)
policies_df=policies_df.drop("ID",axis=0)

In [258]:
policies_df.head()

,Country,Date,PolicyValue,PolicyFlag,PolicyNotes,PolicySource
PolicyType,,,,,,
C1,AD,2020-01-01,0.0,NaN,NaN,OxCGRT
C2,AD,2020-01-01,0.0,NaN,NaN,OxCGRT
C3,AD,2020-01-01,0.0,NaN,NaN,OxCGRT
C4,AD,2020-01-01,0.0,NaN,NaN,OxCGRT
C5,AD,2020-01-01,0.0,NaN,NaN,OxCGRT


In [259]:
policies_df['Policy']=policies_df.index
policies_df.set_index('Date', inplace=True)

In [260]:
policies_df.head()

,Country,PolicyValue,PolicyFlag,PolicyNotes,PolicySource,Policy
Date,,,,,,
2020-01-01,AD,0.0,NaN,NaN,OxCGRT,C1
2020-01-01,AD,0.0,NaN,NaN,OxCGRT,C2
2020-01-01,AD,0.0,NaN,NaN,OxCGRT,C3
2020-01-01,AD,0.0,NaN,NaN,OxCGRT,C4
2020-01-01,AD,0.0,NaN,NaN,OxCGRT,C5


## Policies feature engineering

Check

In [13]:
policies_df[policies_df.Policy=='ID'].PolicyValue.unique()

array([], dtype=float64)

The following policy values are continuous and cannot be categorised (they represent amount of money), we transform them into a power of ten and only keep the exponent to give an idea of the amount at stake.
E3, E4, H4, H5 
the 'I' values are indices, and are dropped from the table

In [14]:
def logify (x):
    if x > 6:
        try:
            if x>0:
                y= int(np.log10(x))
            else:
                y=int(0)
        except:
            y=np.nan
    else:
        try:
            y = int(x)
        except:
            y=np.nan
    return y

In [15]:
pol_df=policies_df

In [16]:
pol_df['PolicyValue']=pol_df['PolicyValue'].apply(logify)
#pol_df.PolicyValue=pol_df.PolicyValue.to_string()

In [17]:
pol_df.head()

,Country,PolicyValue,PolicyFlag,PolicyNotes,PolicySource,Policy
Date,,,,,,
2020-01-01,AD,0.0,NaN,NaN,OxCGRT,C1
2020-01-01,AD,0.0,NaN,NaN,OxCGRT,C2
2020-01-01,AD,0.0,NaN,NaN,OxCGRT,C3
2020-01-01,AD,0.0,NaN,NaN,OxCGRT,C4
2020-01-01,AD,0.0,NaN,NaN,OxCGRT,C5


In [18]:
type(pol_df['PolicyValue'].unique()[3])

numpy.float64

In [19]:
def stringify (x):
    try:
        if x == np.nan:
            y = int(0)
        else:
            y = int(x)
    except:
        y = int(0)
    return str(y)

In [20]:
pol=pol_df
pol.PolicyValue=pol.PolicyValue.apply(stringify)

In [303]:
pol.head()

,Country,PolicyValue,PolicyFlag,PolicyNotes,PolicySource,Policy,TypeVal
Date,,,,,,,
2020-01-01,AD,0,NaN,NaN,OxCGRT,C1,C1_0
2020-01-01,AD,0,NaN,NaN,OxCGRT,C2,C2_0
2020-01-01,AD,0,NaN,NaN,OxCGRT,C3,C3_0
2020-01-01,AD,0,NaN,NaN,OxCGRT,C4,C4_0
2020-01-01,AD,0,NaN,NaN,OxCGRT,C5,C5_0


Check

In [304]:
pol.Policy.unique()

array(['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'E1', 'E2', 'E3',
       'E4', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'I1D', 'I2D', 'I3D',
       'I4D', 'I4L', 'I4LD', 'M1'], dtype=object)

In [305]:
a=pol['Policy'].iloc[1000]+'_'+pol['PolicyValue'].iloc[10000]
a

'C1_2'

In [306]:
p=pol
p['TypeVal']=p['Policy']+'_'+p['PolicyValue']

In [307]:
p.TypeVal.values

array(['C1_0', 'C2_0', 'C3_0', ..., 'I4L_0', 'I4LD_1', 'M1_0'],
      dtype=object)

In [308]:
# Number of features
len(p.TypeVal.unique())

125

In [309]:
q=pd.get_dummies(p.TypeVal)

In [310]:
q['Country']=p['Country']

In [311]:
final_df=pd.DataFrame()
for c in q.Country.unique():
    r=q[q['Country']==c].groupby('Date').sum()
    r['Country']=c
    final_df=pd.concat([final_df,r])

In [312]:
final_df.columns.values

array(['C1_0', 'C1_1', 'C1_2', 'C1_3', 'C2_0', 'C2_1', 'C2_2', 'C2_3',
       'C3_0', 'C3_1', 'C3_2', 'C4_0', 'C4_1', 'C4_2', 'C4_3', 'C4_4',
       'C5_0', 'C5_1', 'C5_2', 'C6_0', 'C6_1', 'C6_2', 'C6_3', 'C7_0',
       'C7_1', 'C7_2', 'C8_0', 'C8_1', 'C8_2', 'C8_3', 'C8_4', 'E1_0',
       'E1_1', 'E1_2', 'E2_0', 'E2_1', 'E2_2', 'E3_0', 'E3_1', 'E3_10',
       'E3_11', 'E3_12', 'E3_2', 'E3_3', 'E3_4', 'E3_5', 'E3_6', 'E3_7',
       'E3_8', 'E3_9', 'E4_0', 'E4_1', 'E4_10', 'E4_11', 'E4_3', 'E4_4',
       'E4_5', 'E4_6', 'E4_7', 'E4_8', 'E4_9', 'H1_0', 'H1_1', 'H1_2',
       'H2_0', 'H2_1', 'H2_2', 'H2_3', 'H3_0', 'H3_1', 'H3_2', 'H4_0',
       'H4_1', 'H4_10', 'H4_11', 'H4_2', 'H4_3', 'H4_4', 'H4_5', 'H4_6',
       'H4_7', 'H4_8', 'H4_9', 'H5_0', 'H5_1', 'H5_2', 'H5_5', 'H5_6',
       'H5_7', 'H5_8', 'H5_9', 'H6_0', 'H6_1', 'H6_2', 'H6_3', 'H6_4',
       'I1D_0', 'I1D_1', 'I1D_2', 'I1D_4', 'I1D_5', 'I2D_0', 'I2D_1',
       'I2D_2', 'I3D_0', 'I3D_1', 'I3D_2', 'I3D_3', 'I3D_4', 'I3D_5',
 

In [313]:
final_df=final_df.drop(['C1_0','C2_0','C3_0','C4_0','C5_0','C6_0',
'C7_0','C8_0','E1_0','E2_0','E3_0','E4_0','H1_0','H2_0',
'H3_0','H4_0','H5_0','H6_0','M1_0','I1D_0','I2D_0','I3D_0',
'I4D_0','I4L_0','I4LD_0'],axis=1)

In [314]:
final_df["Count"] = final_df.sum(axis=1)
final_df=final_df[final_df['Count']!=0]

In [315]:
final_df=final_df.rename(columns={'Country':'ID'})
final_df=final_df.reset_index()

In [316]:
final_df.head()

,Date,C1_1,C1_2,C1_3,C2_1,C2_2,C2_3,C3_1,C3_2,C4_1,...,I4LD_1,I4LD_2,I4LD_3,I4LD_4,I4L_1,I4L_2,I4L_3,I4L_4,ID,Count
0,2020-02-27,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,AD,1
1,2020-03-02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,AD,3
2,2020-03-03,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,AD,3
3,2020-03-04,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,AD,3
4,2020-03-05,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,AD,3


In [317]:
date=[]
for dte in list(final_df['Date']):
    date.append(dte.strftime("%Y-%m-%d"))
final_df['Date']=date

In [318]:
final_df.head()

,Date,C1_1,C1_2,C1_3,C2_1,C2_2,C2_3,C3_1,C3_2,C4_1,...,I4LD_1,I4LD_2,I4LD_3,I4LD_4,I4L_1,I4L_2,I4L_3,I4L_4,ID,Count
0,2020-02-27,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,AD,1
1,2020-03-02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,AD,3
2,2020-03-03,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,AD,3
3,2020-03-04,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,AD,3
4,2020-03-05,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,AD,3


In [319]:
death_df=data_lma[['Date','ID','J-14','J0','J14']]
death_df.set_index('Date',inplace=True)
death_df['Delta14']=((death_df['J14']/death_df['J0'])**(1/13))/((death_df['J0']/death_df['J-14'])**(1/13))
death_df=death_df.reset_index()

/Users/paulcharpentier/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [320]:
death=death_df[['Date','ID','Delta14']]

In [321]:
death.head()

,Date,ID,Delta14
0,2020-02-27,AD,NaN
1,2020-02-28,AD,NaN
2,2020-03-02,AD,NaN
3,2020-03-11,AD,NaN
4,2020-03-13,AD,NaN


In [263]:
#death[death['Date']=='2020-05-11']
#death['Date']=pd.to_datetime(death['Date'])

In [264]:
#death.Date[0]

'2020-02-27'

In [322]:
final=final_df
final=final.reset_index()
df=pd.merge(final,death,how='inner', left_on=['Date','ID'], right_on = ['Date','ID'])
df=df.dropna()

In [323]:
del df['index']

In [324]:
d=df
d=d.reset_index(drop=True)

In [325]:
d.head()

,Date,C1_1,C1_2,C1_3,C2_1,C2_2,C2_3,C3_1,C3_2,C4_1,...,I4LD_2,I4LD_3,I4LD_4,I4L_1,I4L_2,I4L_3,I4L_4,ID,Count,Delta14
0,2020-03-25,0,0,1,0,0,1,1,0,0,...,0,0,0,1,0,0,0,AD,15,0.0
1,2020-03-25,0,0,1,0,0,1,1,0,0,...,0,0,0,1,0,0,0,AD,15,0.0
2,2020-03-27,0,0,1,0,0,1,1,0,0,...,0,0,0,1,0,0,0,AD,16,0.0
3,2020-03-30,0,0,1,0,0,1,1,0,0,...,0,0,0,1,0,0,0,AD,17,0.0
4,2020-03-31,0,0,1,0,0,1,1,0,0,...,0,0,0,1,0,0,0,AD,16,0.0


In [326]:
d=d[d['Delta14']>0]
d=d[d['Delta14']<10]
d=d[d['Delta14']!=1]

In [327]:
X=d.drop(['Date','Count','Delta14','ID'],axis=1)
y=d['Delta14']

5        0.814956
6        0.962765
7        0.971891
8        0.990682
9        0.990682
10       0.996927
11       0.996927
12       1.001495
20       1.001466
21       0.998536
22       0.998536
26       1.010260
27       1.001555
28       0.998982
36       0.977104
37       0.977104
38       0.984850
39       0.974077
40       0.968090
41       0.976054
42       0.973769
43       0.956266
44       0.975699
45       0.992960
46       0.997106
47       0.997751
48       0.999982
49       0.999982
50       0.999982
51       0.998549
           ...   
12171    1.021138
12172    1.021138
12173    1.021138
12174    1.021138
12175    1.035380
12176    1.020163
12177    1.020163
12178    1.020163
12179    1.020163
12180    0.994741
12181    0.996377
12182    0.996377
12183    0.996377
12184    1.001562
12185    0.999990
12186    0.999990
12187    0.998689
12206    0.939527
12207    0.939527
12208    0.978114
12212    1.051708
12213    1.005823
12214    0.983202
12215    0.993431
12216    0